In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [8]:
df=pd.read_csv('whales/train.csv')

In [12]:
import json
with open('whales/train_embeddings.json', 'r') as f:
    embeddings = json.load(f)

In [32]:
image_ids = df['Image'].tolist()
whale_ids = df['whaleID'].tolist()

X = []
Y = []



In [33]:
for image_id, whale_id in zip(image_ids, whale_ids):
    # Get the embedding for the current image ID
    embedding = embeddings.get(str(image_id))
    
    if embedding is not None:
        X.append(embedding)
        Y.append(whale_id)
    else:
        print(f"Warning: No embedding found for image {image_id}")

X = np.array(X)
Y = np.array(Y)


In [35]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
Y = encoder.fit_transform(Y.reshape(-1, 1)).toarray()


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_keras_model(input_shape, num_classes):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(440, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Create the model
num_classes = len(np.unique(Y_train))
input_shape = X_train.shape[1]
model = create_keras_model(input_shape, num_classes)

In [45]:
def train_model(model, X_train, y_train_onehot, X_test, y_test_onehot, epochs=100, batch_size=128):
    history = model.fit(
        X_train,
        y_train_onehot,
        validation_data=(X_test, y_test_onehot),
        epochs=epochs,
        batch_size=batch_size,
        verbose=2
    )
    return history



In [46]:
# Train the model
history = train_model(model, X_train, Y_train, X_test, Y_test)

Epoch 1/100
23/23 - 4s - loss: 72.0541 - accuracy: 0.0024 - val_loss: 6.4547 - val_accuracy: 0.0028 - 4s/epoch - 181ms/step
Epoch 2/100
23/23 - 0s - loss: 8.3309 - accuracy: 0.0072 - val_loss: 6.0822 - val_accuracy: 0.0069 - 264ms/epoch - 11ms/step
Epoch 3/100
23/23 - 0s - loss: 6.3879 - accuracy: 0.0110 - val_loss: 6.0791 - val_accuracy: 0.0069 - 238ms/epoch - 10ms/step
Epoch 4/100
23/23 - 0s - loss: 6.2409 - accuracy: 0.0100 - val_loss: 6.0759 - val_accuracy: 0.0069 - 238ms/epoch - 10ms/step
Epoch 5/100
23/23 - 0s - loss: 6.1775 - accuracy: 0.0103 - val_loss: 6.0730 - val_accuracy: 0.0069 - 222ms/epoch - 10ms/step
Epoch 6/100
23/23 - 0s - loss: 6.1665 - accuracy: 0.0103 - val_loss: 6.0699 - val_accuracy: 0.0069 - 235ms/epoch - 10ms/step
Epoch 7/100
23/23 - 0s - loss: 6.1171 - accuracy: 0.0103 - val_loss: 6.0671 - val_accuracy: 0.0069 - 238ms/epoch - 10ms/step
Epoch 8/100
23/23 - 0s - loss: 6.1171 - accuracy: 0.0096 - val_loss: 6.0642 - val_accuracy: 0.0069 - 235ms/epoch - 10ms/step
E